In [ ]:
!pip install -q condacolab

In [ ]:
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:14
🔁 Restarting kernel...


In [ ]:
!conda --version
!which conda

conda 23.11.0
/usr/local/bin/conda


In [ ]:
!nvidia-smi
!nvcc --version

Fri Aug 16 00:10:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt


%matplotlib inline

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 490, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 490 (delta 149), reused 124 (delta 91), pack-reused 269 (from 1)
Receiving objects: 100% (490/490), 136.70 KiB | 13.67 MiB/s, done.
Resolving deltas: 100% (251/251), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.2 MB/s eta 0:00:00
Installing the rest of the RAPIDS 24.4.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
  Using cached https://pypi.nvidia.com/cudf-cu12/cudf_cu12-24.4.1-cp310-cp310-manylinux_2_28_x86_64.whl (473.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 GB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.8/137.8 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.9/920.9 kB 54.9 MB/s eta 0:00:00
    

In [ ]:
import cudf
import cupy as cp
import sklearn
from sklearn.linear_model import LinearRegression

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
phenotypes = cudf.read_csv('gdrive/My Drive/colabData/phenotypes.csv')
phenotypes.index = phenotypes['Unnamed: 0']
phenotypes.drop(['Unnamed: 0'], axis=1, inplace = True)

In [ ]:
genotypes = cudf.read_csv('gdrive/My Drive/colabData/genotypes.csv')
genotypes.index = genotypes['Unnamed: 0']
genotypes.drop(['Unnamed: 0'], axis=1, inplace = True)

In [ ]:
genotypes_rand = cudf.read_csv('gdrive/My Drive/colabData/genotypes_rand.csv')
genotypes_rand.index = genotypes_rand['Unnamed: 0']
genotypes_rand.drop(['Unnamed: 0'], axis=1, inplace = True)

In [5]:
!gunzip gdrive/Sharedwithme/colabData/r10.3.2.bed.gz -d gdrive/MyDrive/colabData/r10.3.2.bed

gzip: gdrive/Sharedwithme/colabData/r10.3.2.bed.gz: No such file or directory
gzip: gdrive/MyDrive/colabData/r10.3.2.bed.gz: No such file or directory


In [ ]:
# Calculating the first three PC's
def decompose_grm_pca(genotypes, n_comp = 3, verbose = True):
    from sklearn.decomposition import PCA
    pca = PCA(n_components=20)
    decomp = pca.fit_transform(genotypes)
    if verbose: print(f'explained_variances:{pca.explained_variance_ratio_}\
          \ntotal explained var:{sum(pca.explained_variance_ratio_)}' )
    return cudf.DataFrame(decomp, columns = [f'GRM_PC{x}' for x in range(1, (len(pca.explained_variance_ratio_))+1)],
             index=genotypes.index)

# pcs = decompose_grm_pca()

In [ ]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=cp.nan, strategy='mean')
genotypes_rand.dropna(axis = 'index', how = 'all', inplace = True)
genotypes_rand.iloc[:, :] = imp_mean.fit_transform(genotypes_rand).tolist()
pcs = decompose_grm_pca(genotypes = genotypes_rand)

In [ ]:
genotypes

,1:61235370,1:63706676,1:78033225,1:94378103,1:94674775,1:94757435,1:102666585,1:159075582,1:196895135,1:252683863,...,14:67062377,14:69146094,14:69457499,14:72134341,15:55199121,18:15004098,18:24453540,18:29119628,20:4478777,20:4600526
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
001SP00732AAE,2.0,2.0,2.0,2.0,0.0,2.0,2.0,0.0,1.0,2.0,...,1.0,1.0,2.0,1.0,0.0,1.0,2.0,2.0,2.0,0.0
002SP00737AAE,2.0,2.0,2.0,2.0,1.0,2.0,0.0,0.0,2.0,1.0,...,1.0,1.0,2.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0
003SP00744AAE,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,2.0,2.0,0.0,0.0,1.0,1.0,2.0,2.0,1.0
004SP00735AAE,1.0,2.0,2.0,2.0,0.0,2.0,1.0,0.0,1.0,2.0,...,1.0,1.0,2.0,0.0,0.0,0.0,2.0,2.0,2.0,0.0
005SP00752AAE,1.0,2.0,2.0,0.0,2.0,0.0,1.0,0.0,0.0,2.0,...,1.0,1.0,2.0,0.0,1.0,0.0,2.0,2.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46693-1-5,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,...,2.0,1.0,2.0,0.0,1.0,0.0,2.0,2.0,2.0,1.0
48371-1,2.0,2.0,2.0,1.0,1.0,1.0,2.0,0.0,1.0,1.0,...,1.0,1.0,2.0,1.0,0.0,0.0,2.0,2.0,1.0,2.0
44789-2-4,1.0,2.0,2.0,2.0,1.0,2.0,0.0,0.0,1.0,1.0,...,1.0,1.0,2.0,2.0,1.0,0.0,2.0,2.0,1.0,1.0


In [ ]:
grm2 = GRM(genotypes)
grm = SemiDefPosGRM(grm2)

In [ ]:
grm

array([[ 0.83410114, -0.10311954, -0.03384506, ...,  0.05674161,
        -0.01750396,  0.09515813],
       [-0.10311954,  0.74687088,  0.14195132, ...,  0.08946206,
        -0.12527152, -0.1457855 ],
       [-0.03384506,  0.14195132,  0.92773367, ..., -0.07453073,
        -0.0306905 ,  0.12939449],
       ...,
       [ 0.05674161,  0.08946206, -0.07453073, ...,  0.87611398,
         0.27297056,  0.31799933],
       [-0.01750396, -0.12527152, -0.0306905 , ...,  0.27297056,
         1.40961527, -0.06324322],
       [ 0.09515813, -0.1457855 ,  0.12939449, ...,  0.31799933,
        -0.06324322,  4.23637822]])

In [ ]:
# Creating the grm from random snps
# take out x, y and mt snps
rands = sorted(cp.random.randint(0, 5802667, size = 70000))
genotypes_rand = cudf.DataFrame(bed[rands, :].astype(np.float16), columns = fam.iid.values, index = bim.snp[rands].values).T
genotypes_rand

In [ ]:
# Combining the phenotype data with the PC's
pcPheno = pd.concat([phenotypes, pcs], axis=1)
pcPheno

In [ ]:
# send in genotypes for X
def GRM(X, scale = True, return_weights= False, nan_policy = 'ignore', correlation_matrix= False):
    ##### z calculation
    x = X.to_cupy(na_value = 0)
    z = x - cp.nanmean(x, axis = 0)
    if scale: z /=  cp.nanstd(x, axis = 0)
    cp.nan_to_num(z, copy = False,  nan=0.0, posinf=None, neginf=None )
    zzt = cp.dot(z,z.T)
    if nan_policy == 'mean': zzt_w = x.shape[1]-1
    #### NA adjustment
    elif nan_policy in ['ignore', 'per_iid']:
        zna = (~cp.isnan(x)).astype(cp.float32)
        zzt_w = cp.dot(zna,zna.T)
        zzt_w = cp.clip(zzt_w-1, a_min = 1, a_max = cp.inf)
        if nan_policy == 'per_iid':
            zzt_w = zzt_w.max(axis =1)[:, None]
    grm = zzt/zzt_w
    if correlation_matrix:
        sig = cp.sqrt(cp.diag(grm))
        grm /= cp.outer(sig, sig)
        cp.fill_diagonal(grm, 1)
    if return_weights:
        return {'zzt': zzt, 'weights': zzt_w, 'grm': grm }
    else:
        return grm

In [ ]:
#deals with weird missing values in the grm run after grm has been created

def SemiDefPosGRM(grm, eps = 1e-3):
    # grm = grm.to_cupy(na_value = 0)
    try:
        cp.linalg.cholesky(grm)
        return grm
    except:
        eigvals, eigvecs = cp.linalg.eigh(grm)
        # Set any negative eigenvalues to zero
        eigvals[eigvals < 0] = 0
        # Reconstruct the matrix
        return eigvecs @ cp.diag(eigvals) @ eigvecs.T + eps*cp.eye(grm.shape[0])

In [ ]:
# run to get the lambda value for blup
# grm is variance

def H2og(y, grm):
    y = cp.array(y).flatten()
    obs = cp.where(~cp.isnan(y))[0]
    y = y[obs]
    grm = cp.array(grm)
    grm = grm[obs.reshape(-1,1), obs]
    from scipy.optimize import minimize
    def _ML(h2):
        sp = cp.nanvar(y)
        sg = h2*sp
        se = sp - sg
        m = cp.nanmean(y)*cp.ones(y.shape[0])
        cov = sg*grm + se*cp.eye(y.shape[0])
        return -mvn(mean = m, cov=cov).logpdf(y)
    return minimize(fun = _ML, x0 = .5, bounds=[(0.,1.)]).x[0]
# %time H2og(tempdf[['regressedlr_rt_peropfalsealarm']], G0)


# run the second one, much faster

from scipy.optimize import minimize
from scipy.linalg import cholesky, solve_triangular
from scipy.stats import multivariate_normal as mvn
def H2(y, grm, l = 'ML'):
    """
    Estimate heritability (h2) using maximum likelihood estimation with Cholesky decomposition.

    :param y: Phenotypic values (numpy.ndarray), shape (N,) or (N, T)
    :param grm: Genetic Relationship Matrix (numpy.ndarray), shape (N, N)
    :return: Estimated heritability (h2)
    """
    y = cp.array(y).flatten()
    grm = cp.array(grm)
    obs = cp.where(~cp.isnan(y))[0]
    y = y[obs]
    grm = cp.array(grm)
    grm = grm[obs.reshape(-1,1), obs]
    N = y.shape[0]
    def _L(h2):
        sp = cp.nanvar(y, axis=0, ddof=1)  # Phenotypic variance
        sg = h2 * sp  # Genetic variance
        se = sp - sg  # Residual variance
        m = cp.nanmean(y, axis=0)  # Mean of y
        cov = sg * grm + se * cp.eye(grm.shape[0])
        L = cholesky(cov, lower=True)  # Cholesky decomposition
        # Log likelihood calculation using Cholesky decomposition
        alpha = solve_triangular(L, y - m, lower=True)
        log_det = 2.0 * cp.sum(cp.log(cp.diag(L)))
        log_likelihood = -0.5 * (cp.dot(alpha.T, alpha) + log_det + y.shape[0] * cp.log(2 * cp.pi))
        if l == 'REML':
            X = cp.ones((N, 1))
            XtX_inv = cp.linalg.inv(X.T @ X)
            P = cp.eye(N) - X @ XtX_inv @ X.T
            P_log_det = cp.linalg.slogdet(P)[1]  # log determinant of P
            log_likelihood = log_likelihood - 0.5 * P_log_det
        return -log_likelihood
    result = minimize(fun=_L, x0=0.5, bounds=[(0., 1.)])
    print(result)
    return result.x[0]
# %time H2(tempdf[['regressedlr_rt_peropfalsealarm']], G0, l = 'ML')

In [ ]:
# linear mixed model using numpy

# p = 'regressedlr_soc_midconrs_16_5'

for p in phenotypes.columns:
    print(p)

    grm_columns = pcPheno.filter(regex='GRM_PC', axis=1).columns.tolist()

    # concatenating the pc's and the trait column
    pcp = cudf.concat([pcPheno[grm_columns], pcPheno[p]], axis = 1)

    # display(pcp[p].value_counts())

    #dropping all rows with na values
    pcp = pcp.dropna(subset = grm_columns + [p], how = 'any')

    display(pcp)

    #drop the trait if the dropping of na's has left an empty dataframe
    if len(pcp.index) == 0:
        continue
    else:
        # covariates
        c = cp.ones(shape = [pcp.shape[0], 1])
        # display(c)

        #pc's
        X = cp.array(pcp.filter(regex='GRM_PC', axis=1))
        # print(np.isna(X).sum())
        # display(X)

        # should be sigma g/sigma e

        h2 = H2(pcp[p], grm, l = 'ML')

        print(h2)

        λ = (1-h2)/h2

        # traits
        y = cp.array(pcp.loc[:, p])

        #each of the four corners of the first matrix
        ul = c.T @ c

        ur = c.T @ X

        ll = X.T @ c

        lr = X.T @ X + (cp.eye(X.shape[1]) * λ)

        upper = cp.concatenate((ul, ll), axis = 0)

        lower = cp.concatenate((ur, lr), axis = 0)

        #first matrix that will be inversed in the final calculation
        matrix = cp.concatenate((upper, lower), axis = 1)

        #second matrix
        up = c.T @ y
        down = X.T @ y

        secondMatrix = cp.concatenate((up, down), axis = 0)
        secondMatrix = secondMatrix[:, None]

        bigCalc = cp.linalg.inv(matrix) @ secondMatrix

        u_hat_value = bigCalc[0] #fixed effects
        alpha_hat = bigCalc[1:] #random effects

        y_hat = X @ alpha_hat + u_hat_value

        adjusted_phenotypes = y-y_hat
        print(adjusted_phenotypes)
        adjusted_phenotype_results.join(adjusted_phenotypes)
        print(adjusted_phenotypes_results)
